# agent search engine

In [1]:
import sys
sys.path.append("../")
from setup import trace_ai21
from langchain.tools import DuckDuckGoSearchRun

search = DuckDuckGoSearchRun()

MODEL = trace_ai21('food_search')

Enable tracing at food_search
AI21 ready


In [2]:
ans = search("What are the top signature dishes in north Vietnam?")
ans

'Bún bò Huế, a spicy, lemongrass rice vermicelli noodle soup served with fresh herbs and vegetables Vietnamese cuisine encompasses the foods and beverages of Vietnam. Meals feature a combination of five fundamental tastes ( ngũ vị ): sweet, salty, bitter, sour, and spicy. 1. Banh Mi Originating from France, the Banh Mi (Vietnamese baguette) is made from a long list of ingredients that might make you feel overstuffed when you first hear of it, but the harmonious combinations will change your thoughts. The baguette with the crispy crust is filled with pate, butter, various meat types, and fresh vegetables. Top Traditional Vietnamese Dishes You Need to Try ASIA / VIETNAM Bánh mì is a traditional Vietnamese baguette sandwich | © Hong Hanh Mac Thi / Alamy Stock Photo Isabelle Sudron 12 September 2022 View Mì Quảng We love Mì Quảng. For great value, it\'s the perfect Vietnamese meal — noodles, peanuts, rice crackers, pork and a turmeric broth. This dish is the pride of Quảng Nam Province in 

# test chain

In [23]:
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from setup import trace_openai

MODEL = trace_openai("food_search")

TEMPLATE = """ Given a paragraph about Vietnam's signature dishes.
Please give short description for each dish.
The paragraph: {para}
The foods:"""

prompt = PromptTemplate(input_variables=['para'], template=TEMPLATE)

chain = LLMChain(llm=MODEL, prompt=prompt)

Enable tracing at food_search
OPENAI ready


In [29]:
foods = chain.run(ans)
foods

'\n\nCurry Udon: A Japanese dish consisting of thick noodles made from wheat flour, served in a bowl of Japanese curry.\nShabu-shabu: A popular Japanese dish consisting of thinly sliced meat and vegetables cooked in water, evolved from the traditional Mongolian nabemono (one-pot) cooking.\nYakisoba: Japanese fried noodles usually mixed with a variety of meat and vegetables, such as pork, cabbage, onions, and carrots.\nKaiyaki-misu: A traditional Japanese dish originating from the Tsugaru region in Aomori prefecture, made by simmering scallops, eggs, leeks, and scallions in miso-based broth.\nChicken Katsu: A traditional Japanese dish and a type of katsu, made by dredging tender chicken breast in flour, eggs, and panko breadcrumbs, then frying the cutlets in hot oil until they become golden and crunchy.'

# agent

In [27]:
from langchain.agents import initialize_agent
from langchain.agents import AgentType
from langchain.tools import Tool

AI21 = trace_ai21('food_search')

search = DuckDuckGoSearchRun()

search_tool = Tool.from_function(func=search.run, name="Search", description="useful for when you need to answer questions about signature dishes.")

agent = initialize_agent([search_tool], AI21, agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION, verbose=True)

Enable tracing at food_search
AI21 ready


# BUILD

In [3]:
import sys
sys.path.append("../")
from langchain.tools import DuckDuckGoSearchRun
from langchain.llms import BaseLLM
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
# my code
from setup import trace_openai, trace_ai21
from utils.prompt_utils import build_prompt
from utils.json_parser import parse_nested_json

def build_list_chain(MODEL: BaseLLM) -> LLMChain:
    inputs = ['para']
    outputs = {"chain": "always return 'find_food'",
               "food": "a js array of signature dishes.",
               "desc": "a js array of signature dish descriptions"}
    TEMPLATE = """ Given a paragraph about Vietnam's signature dishes delimited by triple backquotes.
    ```
    {para}
    ```
    Please do your best to complete the task as instructed.
    {format_instructions}
    Answer:"""
    
    prompt = build_prompt(inputs, outputs, TEMPLATE)
    chain = LLMChain(llm=MODEL, prompt=prompt, output_key='result')
    return chain

class FoodFinder:

    def __init__(self, model: BaseLLM):
        self.search_prompt = "What are the top signature dishes in {country}?"
        self.search_tool = DuckDuckGoSearchRun()
        self.chain = build_list_chain(model)
        print("Food finder is ready")

    def find(self, place: str):
        paragraph = self.search_tool(place)
        ans = self.chain({"para": paragraph})
        return parse_nested_json(ans['result'])

In [4]:
MODEL = trace_ai21("food_search", 1000)

food_finder = FoodFinder(MODEL)

data = food_finder.find("south Vietnam")

Enable tracing at food_search
AI21 ready
Food finder is ready


In [5]:
data

{'chain': 'find_food',
 'food': ['pho',
  'bun cha',
  'banh mi',
  'nem',
  'bun thit nuong',
  'banh xeo',
  'bun bo hue',
  'banh cuon',
  'chao tom',
  'banh khot',
  'banh trang nuong',
  'banh uot',
  'banh khoai',
  'banh bot loc',
  'banh cuon',
  'banh uot',
  'banh khoai',
  'banh bot loc'],
 'desc': ['Pho is a Vietnamese soup consisting of broth, rice noodles, herbs, and meat. It is a popular street food in Vietnam and is often eaten for breakfast.',
  'Bun cha is a Vietnamese dish consisting of grilled pork, rice noodles, and herbs. It is a popular street food in Vietnam and is often eaten for breakfast.',
  'Banh mi is a Vietnamese sandwich consisting of pork, vegetables, and mayonnaise. It is a popular street food in Vietnam and is often eaten for breakfast.',
  'Nem is a Vietnamese dish consisting of pork, vegetables, and noodles. It is a popular street food in Vietnam and is often eaten for breakfast.',
  'Bun thit nuong is a Vietnamese dish consisting of grilled pork, 